In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [3]:
data_path = "mle-training/data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4518,9.0,0.200,0.33,3.5,0.049,10.0,40.0,0.99440,3.14,0.36,9.8,6
3424,6.4,0.320,0.27,4.9,0.034,18.0,122.0,0.99160,3.36,0.71,12.5,6
819,6.6,0.560,0.22,8.9,0.034,27.0,133.0,0.99675,3.20,0.51,9.1,5
4479,6.0,0.555,0.26,4.5,0.053,17.0,126.0,0.99430,3.24,0.46,9.1,5
2966,6.6,0.230,0.30,14.9,0.051,33.0,118.0,0.99835,3.04,0.54,9.0,6
3662,6.1,0.600,0.12,1.8,0.050,11.0,76.0,0.99268,3.42,0.48,10.4,4
1171,8.2,0.610,0.45,5.4,0.030,15.0,118.0,0.99540,3.14,0.34,9.6,5
1179,7.2,0.200,0.25,4.5,0.044,31.0,109.0,0.99490,3.23,0.36,9.4,5
1461,6.4,0.145,0.49,5.4,0.048,54.0,164.0,0.99460,3.56,0.44,10.8,6
2920,7.4,0.280,0.40,11.9,0.032,13.0,92.0,0.99629,3.01,0.46,10.8,4


In [4]:
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [5]:
mlflow.tracking.get_tracking_uri()


'http://0.0.0.0:5000'

In [7]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2022/06/09 19:04:52 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [10]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "mle-training/data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [11]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772677
Save to: mlruns/1/bbd46c6fb9124bc481843b0f8abedf39/artifacts


In [12]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093895
  R2: 0.20224631822892103
Save to: mlruns/1/3a31ecc6aeed474a97d30a2d30cffb80/artifacts


In [13]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlruns/1/f4aed08da86a4bfe801ee69f243f0e82/artifacts
